# Final CHO Model
This notebook is to asses the validity of our reconstruction and how complete it is.

[1. Generation of the dataset and model reconstruction](#generation) <br>
&nbsp;&nbsp;&nbsp;&nbsp;**1.1 Retrieve information from the Google Sheet datasets reactions and metabolites**<br>
&nbsp;&nbsp;&nbsp;&nbsp;**1.2 Build a model and feed it the information from the df generated** <br>
&nbsp;&nbsp;&nbsp;&nbsp;**1.3 Save and validate the model** <br>
&nbsp;&nbsp;&nbsp;&nbsp;**1.4 Check for unbalanced reactions** <br>

[2. Identification of Blocked Reactions and Dead-End Metabolites](#blocked&deadends) <br>
&nbsp;&nbsp;&nbsp;&nbsp;**2.1 Identification of Blocked Reactions**<br>
&nbsp;&nbsp;&nbsp;&nbsp;**2.2 Identification of Dead-Ends Metabolites** <br>
&nbsp;&nbsp;&nbsp;&nbsp;**2.3 Gap-fill** <br>
&nbsp;&nbsp;&nbsp;&nbsp;**2.4 Addition of Extracellular Exchange Reanctions** <br>

[3. Generation of the Mass Flow Graph](#MFG) <br>
&nbsp;&nbsp;&nbsp;&nbsp;**3.1 Generation of the "D-Matrix"**<br>
&nbsp;&nbsp;&nbsp;&nbsp;**3.2 Plotting the "D-Matrix", Normalized Flow Graph (NFG)** <br>
&nbsp;&nbsp;&nbsp;&nbsp;**3.3 Generation of the "FluxOpenValue" matrix** <br>
&nbsp;&nbsp;&nbsp;&nbsp;**3.4 Generation of the "Mass Flow Graph (MFG) Matrix"** <br>
&nbsp;&nbsp;&nbsp;&nbsp;**3.5 Plotting the "MFG Matrix"** <br>

[4. Biomass Conecting Reactions](#biomass) <br>

## 1. Generation of the dataset and model reconstruction <a id='generation'></a>
Here we generate the CHO model from the dataset stored in the Google Sheet file. We first use the google_sheet module to extract all the necessary information from the original dataset. Then we use those dataset and the COBRApy library to: (1) Create a new model and add reactions from the **Rxns Sheet**, (2) Add information on each reaction obtained from the **Rxns Sheet** and **Attributes Sheet**, (3) Add boundary reactions from the **BoundaryRxns Sheet**, and (4) Add information for each metabolite from the **Metabolites Sheet**. Finally we save the model as a SBML file and validate it using the cobrapy built-in function "validate_sbml_model( )".

In [1]:
# Import libraries
import pandas as pd
import numpy as np

import cobra
from cobra import Model, Reaction, Metabolite
from cobra.io import validate_sbml_model, save_json_model, write_sbml_model, save_matlab_model, load_matlab_model

from tqdm.notebook import tqdm

from google_sheet import GoogleSheet

### 1.1 Retrieve information from the Google Sheet datasets reactions and metabolites

In [2]:
##### ----- Generate datasets from Google Sheet ----- #####

#Credential file
KEY_FILE_PATH = 'credentials.json'

#CHO Network Reconstruction + Recon3D_v3 Google Sheet ID
SPREADSHEET_ID = '1MlBXeHIKw8k8fZyXm-sN__AHTRSunJxar_-bqvukZws'

# Initialize the GoogleSheet object
sheet = GoogleSheet(SPREADSHEET_ID, KEY_FILE_PATH)

# Read data from the Google Sheet
sheet_met = 'Metabolites'
sheet_rxns = 'Rxns'
sheet_attributes = 'Attributes'
sheet_boundary = 'BoundaryRxns'
sheet_genes = 'Genes'

metabolites = sheet.read_google_sheet(sheet_met)
rxns = sheet.read_google_sheet(sheet_rxns)
rxns_attributes = sheet.read_google_sheet(sheet_attributes)
boundary_rxns = sheet.read_google_sheet(sheet_boundary)
genes_df = sheet.read_google_sheet(sheet_genes)

### 1.2 Build a model and feed it the information from the df generated

In [3]:
##### ----- Create a model and add reactions ----- #####
model = Model("iCHO3644")
lr = []
for _, row in rxns.iterrows():
    r = Reaction(row['Reaction'])
    lr.append(r)    
model.add_reactions(lr)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-04


In [4]:
##### ----- Add information to each one of the reactions ----- #####
for i,r in enumerate(tqdm(model.reactions)):
    r.build_reaction_from_string(rxns['Reaction Formula'][i])
    r.name = rxns['Reaction Name'][i]
    r.subsystem = rxns['Subsystem'][i]
    if not (pd.isna(rxns['GPR_final'][i]) or rxns['GPR_final'][i] == ''):
        r.gene_reaction_rule = str(rxns['GPR_final'][i])
    r.lower_bound = float(rxns_attributes['Lower bound'][i])
    r.upper_bound = float(rxns_attributes['Upper bound'][i])
    r.annotation['confidence_score'] = str(rxns['Conf. Score'][i])

  0%|          | 0/10044 [00:00<?, ?it/s]

unknown metabolite 'ala_L_m' created
unknown metabolite 'glx_m' created
unknown metabolite 'gly_m' created
unknown metabolite 'pyr_m' created
unknown metabolite 'ala_L_x' created
unknown metabolite 'glx_x' created
unknown metabolite 'gly_x' created
unknown metabolite 'pyr_x' created
unknown metabolite 'argsuc_c' created
unknown metabolite 'arg_L_c' created
unknown metabolite 'fum_c' created
unknown metabolite 'asp_L_c' created
unknown metabolite 'atp_c' created
unknown metabolite 'citr_L_c' created
unknown metabolite 'amp_c' created
unknown metabolite 'h_c' created
unknown metabolite 'ppi_c' created
unknown metabolite 'asn_L_c' created
unknown metabolite 'h2o_c' created
unknown metabolite 'nh4_c' created
unknown metabolite 'asn_L_m' created
unknown metabolite 'h2o_m' created
unknown metabolite 'asp_L_m' created
unknown metabolite 'nh4_m' created
unknown metabolite 'gln_L_c' created
unknown metabolite 'glu_L_c' created
unknown metabolite 'accoa_m' created
unknown metabolite 'Nacasp_m' c

In [5]:
##### ----- Add Boundary Reactions ----- #####
dr = []
for _, row in boundary_rxns.iterrows():
    r = Reaction(row['Reaction'])
    dr.append(r)    
model.add_reactions(dr)

boundary_rxns_dict = boundary_rxns.set_index('Reaction').to_dict()
boundary_rxns_dict

for i,r in enumerate(tqdm(model.reactions)):
    if r in dr:
        r.build_reaction_from_string(boundary_rxns_dict['Reaction Formula'][r.id])
        r.name = boundary_rxns_dict['Reaction Name'][r.id]
        r.subsystem = boundary_rxns_dict['Subsystem'][r.id]
        r.lower_bound = float(boundary_rxns_dict['Lower bound'][r.id])
        r.upper_bound = float(boundary_rxns_dict['Upper bound'][r.id])
        r.annotation['confidence_score'] = str(1)
model

  0%|          | 0/11010 [00:00<?, ?it/s]

Name,iCHO3644
Memory address,1496a5870
Number of metabolites,7345
Number of reactions,11010
Number of genes,3644
Number of groups,0
Objective expression,0
Compartments,


In [6]:
##### ----- Add information for each metabolite ----- #####
metabolites_dict = metabolites.set_index('BiGG ID').to_dict('dict')
for met in model.metabolites:
    met.name = metabolites_dict['Name'][f'{met}']
    met.formula = metabolites_dict['Formula'][f'{met}']
    met.compartment = metabolites_dict['Compartment'][f'{met}'].split(' - ')[0]
    try:
        met.charge = int(metabolites_dict['Charge'][f'{met}'])
    except (ValueError, TypeError):
        print(f'{met} doesnt have charge')

dnac_n doesnt have charge
lipACP_c doesnt have charge


In [7]:
##### ----- Add Gene Name information ----- #####
genes_dict = genes_df.iloc[:,:2].set_index('Gene Entrez ID').to_dict('dict')
for g in model.genes:
    if g.id in list(genes_dict['Gene Symbol'].keys()):
        g.name = genes_dict['Gene Symbol'][f'{g}']

### 1.3 Save and validate the model

In [8]:
##### ----- Build the S matrix ----- #####
S = cobra.util.create_stoichiometric_matrix(model, array_type='dense')
model.S = S

model.solver = 'gurobi'
model.objective = 'biomass_cho'
sol = model.optimize()
sol

,fluxes,reduced_costs
AGTim,0.0,0.0
AGTix,0.0,0.0
ARGSL,0.0,0.0
ARGSS,0.0,0.0
ASNN,0.0,0.0
...,...,...
EX_CE2516_e,0.0,0.0
EX_pcresol_e,0.0,0.0
EX_pcs_e,0.0,0.0
EX_indole_e,0.0,0.0


In [9]:
c=0
for f in sol.fluxes:
    if f != 0:
        c+=1
        
print(c)

440


In [10]:
##### ----- Save the model ----- #####

# XML
model_name_xml = 'iCHO3644.xml' 
write_sbml_model(model, model_name_xml)

# JSON, because the sbml doesnt save the subsystems
model_name_json = 'iCHO3644.json' 
save_json_model(model, model_name_json)

# MATLAB
model_name_matlab = 'iCHO3644.mat' 
save_matlab_model(model, model_name_matlab)

In [11]:
##### ----- Test for errors in the recostruction ----- ######

# import tempfile
# from pprint import pprint
# from cobra.io import write_sbml_model, validate_sbml_model
# with tempfile.NamedTemporaryFile(suffix='.xml') as f_sbml:
#     write_sbml_model(model, filename=f_sbml.name)
#     report = validate_sbml_model(filename=f_sbml.name)
# pprint(report)

from cobra.io import read_sbml_model, validate_sbml_model
(_, errors) = validate_sbml_model(model_name_xml)
errors

{'SBML_FATAL': [],
 'SBML_ERROR': [],
 'SBML_SCHEMA_ERROR': [],
 'SBML_WARNING': [],
 'COBRA_FATAL': [],
 'COBRA_ERROR': [],
 'COBRA_WARNING': [],
 'COBRA_CHECK': []}

### 1.4 Check for unbalanced reactions
Once the model is checked and saved as a xml and json format we then evaluate the amount of mass and charge unbalanced reactions

In [12]:
# Check for unbalanced reactions
subsystems = ['BIOMASS', 'PROTEIN ASSEMBLY', 'PROTEIN DEGRADATION'] # filter out the reactions from these subsystems

# Lists to store the data for each column
reaction_ids = []
formulas = []
metabolites = []
unbalances = []

counter = 0
for rxn in model.reactions:
    if not rxn.id.startswith(('EX_','DM_','SK_')) and rxn.subsystem not in subsystems:
        mb = rxn.check_mass_balance()
        if mb != {}:# and set(mb.keys()) != {'charge'}:  # Check if dictionary has keys other than 'charge'
            counter+=1
            prod_ids = [{met.id:met.formula} for met in rxn.products]
            react_ids = [{met.id:met.formula} for met in rxn.reactants]
            # Append values to lists
            print(rxn.id)
            reaction_ids.append(rxn.id)
            print(rxn.reaction)
            formulas.append(rxn.reaction)
            print(react_ids + prod_ids)
            metabolites.append(react_ids + prod_ids)
            print(mb)
            unbalances.append(mb)
            print('...............................')
print(counter)

# Create DataFrame from lists
mass_unbalanced_reactions = pd.DataFrame({
    "Reaction ID": reaction_ids,
    "Formula": formulas,
    "Metabolites": metabolites,
    "Unbalance": unbalances
})

mass_unbalanced_reactions.to_excel("temp/mass_unbalanced_reactions.xlsx", engine='openpyxl', index=False)

r0145
2.0 arg_L_c + 3.0 nadph_c + 4.0 o2_c --> 2.0 citr_L_c + 4.0 h2o_c + 3.0 nadp_c + 2.0 no_c
[{'arg_L_c': 'C6H15N4O2'}, {'nadph_c': 'C21H26N7O17P3'}, {'o2_c': 'O2'}, {'citr_L_c': 'C6H13N3O3'}, {'h2o_c': 'H2O'}, {'nadp_c': 'C21H25N7O17P3'}, {'no_c': 'NO'}]
{'charge': 1.0, 'H': 1.0}
...............................
UNK2
2kmb_c + gln_L_c + 2.0 h_c --> glu_L_c + met_L_c
[{'2kmb_c': 'C5H7O3S'}, {'gln_L_c': 'C5H10N2O3'}, {'h_c': 'H'}, {'glu_L_c': 'C5H8NO4'}, {'met_L_c': 'C5H11NO2S'}]
{'charge': -2.0}
...............................
GAUGE-R04283
4izp_c + 3.0 h2o_c <=> 4ogm_c + for_c + h_c + nh4_c
[{'4izp_c': 'C6H7N2O3'}, {'h2o_c': 'H2O'}, {'4ogm_c': 'C5H6NO4'}, {'for_c': 'CH1O2'}, {'nh4_c': 'H4N'}, {'h_c': 'H'}]
{'charge': 1.0, 'H': -1.0}
...............................
r0001
ametam_c --> 5mta_c + HC02119_c
[{'ametam_c': 'C14H24N6O3S'}, {'5mta_c': 'C11H15N5O3S'}, {'HC02119_c': 'X'}]
{'charge': -2, 'C': -3, 'H': -9, 'N': -1, 'X': 1}
...............................
CYOOm3
4.0 focytC_m + 7.92 

In [ ]:
# Update the "Balance status" column based on whether the reaction is present in "reaction_ids"
rxns_copy = rxns.copy()
rxns_copy['Balance status'] = rxns_copy['Reaction'].apply(lambda x: 'UNBALANCED' if x in reaction_ids else 'BALANCED')
rxns_equals = rxns_copy.equals(rxns)

In [ ]:
##############################################
#### ------------------------------------ ####
#### ---- Update Rxns Google Sheets ----- ####
#### ------------------------------------ ####
##############################################
if not rxns_equals:
    sheet.update_google_sheet(sheet_rxns, rxns_copy)
    print("Google Sheet updated.")

## 2. Identification of Blocked Reactions and Dead-End Metabolites <a id='blocked&deadends'></a>
In this second part of the notebook we use two different functions from the utils module to: (1) Run a flux variability analysis and identify blocked reactions, and (2) identify dead-end metabolites. Finally we add Extracellular Exchange reactions for the dead-end metabolites that are in the extracellular compartment.

In [13]:
import pandas as pd
from cobra.io import load_json_model, read_sbml_model, load_matlab_model
from cobra.flux_analysis import find_blocked_reactions, flux_variability_analysis
from utils import detect_dead_ends

In [14]:
##### ----- Read Model ----- #####
if 'model' not in locals():
    model = load_json_model("iCHO3644.json")
    print('Model loaded')
else:
    print('Model already generated')

Model already generated


### 2.1 Identification of Blocked Reactions
Here we use the COBRApy built-in functions **find_blocked_reactions** and **flux_variability_analysis** to find blocked reactions in our reconstruction and run an FVA analysis respectively.

In [ ]:
##### ----- Blocked Reactions ----- #####
for rxn in model.boundary:
    if rxn.id.startswith("EX_"):
        rxn.bounds = (-1000,1000)
    if rxn.id.startswith("SK_"):
        rxn.bounds = (-1000,1000)
    if rxn.id.startswith("DM_"):
        rxn.bounds = (0,1000)

model.solver = 'gurobi'
blocked_reactions = find_blocked_reactions(model)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-04
Read LP format model from file /var/folders/_x/tfg8s2ks4n1ftkkwzp5sqjpc0000gn/T/tmphohybhd5.lp
Reading time = 0.09 seconds
: 7346 rows, 22021 columns, 93411 nonzeros
Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-04
Read LP format model from file /var/folders/_x/tfg8s2ks4n1ftkkwzp5sqjpc0000gn/T/tmp7jhvzgph.lp
Reading time = 0.09 seconds
: 7346 rows, 22021 columns, 93411 nonzeros
Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-04
Read LP format model from file /var/folders/_x/tfg8s2ks4n1ftkkwzp5sqjpc0000gn/T/tmp4pmvshef.lp
Reading time = 0.08 seconds
: 7346 rows, 22021 columns, 93411 nonzeros
Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-04
Read LP format model from file /var/folders/_x/tfg8s2ks4n1ftkkwzp5sqjpc0000gn/T/tmps77u590y.lp
Reading time = 0.09 seconds
: 7346 rows, 22

In [ ]:
### ---- Remove blocked reactions from the model and save it as a separete model ---- ####

model_unblocked = model.copy()

# Convert list of reaction IDs to reaction objects
blocked_reaction_objects = [model_unblocked.reactions.get_by_id(rxn_id) for rxn_id in blocked_reactions]

# Remove blocked reactions
model_unblocked.remove_reactions(blocked_reaction_objects, remove_orphans=True)
print(f"Removed {len(blocked_reaction_objects)} blocked reactions from the model.")

# XML
model_name_xml = 'iCHO3644_unblocked.xml' 
write_sbml_model(model_unblocked, model_name_xml)

# JSON, because the sbml doesnt save the subsystems
model_name_json = 'iCHO3644_unblocked.json' 
save_json_model(model_unblocked, model_name_json)

# MATLAB
model_name_matlab = 'iCHO3644_unblocked.mat' 
save_matlab_model(model_unblocked, model_name_matlab)

In [ ]:
# Save confidence score for Context-Specific model generation

conf_scores = []
for r in model_unblocked.reactions:
    conf_scores.append(r.annotation['confidence_score'])
conf_scores_array = np.array(conf_scores)

np.savetxt("../Data/Context_specific_models/confidence_scores.csv", conf_scores_array, delimiter=",", fmt='%s')

In [ ]:
### ---- FVA ---- ####
model.solver = 'gurobi'
fva_results = flux_variability_analysis(model)
fva_results.to_excel('temp/fva_results.xlsx')

In [ ]:
## Check if any reactions in the BIOMASS subsystem is blocked
for rxn in blocked_reactions:
    r = model.reactions.get_by_id(rxn)
    if r.subsystem == 'BIOMASS':
        print(r.id)
        print('-----------------')
        print('-----------------')
        for met in r.metabolites:
            m = model.metabolites.get_by_id(met.id)
            print(m)
            print('.................')
            for r2 in m.reactions:
                if r2.id in blocked_reactions:
                    print(f'No Flux -> {r2.id}: {r2.reaction}')
                else:
                    print(f'With Flux -> {r2.id}: {r2.reaction}')
            print('.................')
            print(' ')
                

In [ ]:
##### ----- Print the amount  and % of blocked reactions ----- #####
print('##### ----- Blocked Reactions ----- #####')
print(f'The model has {len(model.reactions)} total reactions')
print(f'The model has {len(blocked_reactions)} ({round(len(blocked_reactions)/len(model.reactions)*100)}%) blocked reactions')

### 2.2 Identification of Dead-Ends Metabolites
The detect_dead_ends( ) function from the utils module returns a list with all the **dead-end** metabolites in our model. A dead-end metabolite refers to a metabolite that is either only consumed but not produced, or only produced but not consumed, in a given metabolic network. The results are stored in the "Dead-ends.txt" file.

In [ ]:
##### ----- Detect Dead-Ends ----- #####
model.solver = 'gurobi' #change 'gurobi' for the default cobrapy solver 'glpk' 
dead_ends = detect_dead_ends(model)

In [ ]:
counter=0
for i, is_dead_end in enumerate(dead_ends):
    if is_dead_end:
        metabolite = model.metabolites[i]
        counter+=1
        
print(counter)

In [ ]:
# Save dead ends and their associated reaction in a pandas df
data = []

for i, is_dead_end in enumerate(dead_ends):
    if is_dead_end:
        metabolite = model.metabolites[i]
        reactions = [str(met_rxn) for met_rxn in metabolite.reactions]  # Convert reactions to strings
        data.append([metabolite.id] + reactions)

# Convert the list to a DataFrame
dead_ends_df = pd.DataFrame(data, columns=['Metabolite', 'Reaction1', 'Reaction2', 'Reaction3', 'Reaction4'])  # 'etc.' is a placeholder

# Adjusting the DataFrame to handle variable number of reactions
dead_ends_df = dead_ends_df.apply(lambda x: pd.Series(x.dropna().values), axis=1).fillna('')

# Renaming the columns appropriately
new_columns = ['Metabolite'] + [f'Reaction{i}' for i in range(1, len(dead_ends_df.columns))]
dead_ends_df.columns = new_columns

dead_ends_df.to_excel('temp/dead_ends_reactions.xlsx', index=False)

print(f'Total amount of dead-end metabolites: {len(dead_ends_df)}')  # To display the first few rows of the DataFrame

### 2.3 Gap-fill
Here we try different gap-filling approaches to fix dead-end metabolites in our reconstruction. First, we create artificial transport reactions if we detect that a dead-end metabolites is at oposite sides in different reactions. Then we extract boundary reactions from other reconstructions associated with our dead-end metabolites

In [ ]:
##### --- Create transport reactions to fill the gaps --- ######

def check_metabolite_sides(reactions, metabolite_base):
    sides = []  # List to store the side ('left' or 'right') of each reaction
    for reaction in reactions:
        # Splitting the reaction string into reactants and products
        if '-->' in reaction:
            reactants, products = reaction.split('-->')
        elif '<=>' in reaction:
            reactants, products = reaction.split('<=>')

        # Splitting reactants and products into individual metabolites and trimming whitespace
        reactant_ids = [r.strip() for r in reactants.split('+')]
        product_ids = [p.strip() for p in products.split('+')]

        # Constructing specific identifiers for comparison
        metabolite_id_with_compartment = f"{metabolite_base}_"

        # Checking if the specific identifier is in reactants or products
        if any(metabolite_id_with_compartment in r for r in reactant_ids):
            sides.append('left')
        if any(metabolite_id_with_compartment in p for p in product_ids):
            sides.append('right')

    return 'left' in sides and 'right' in sides


# Use the modified metabolites_compartments dictionary creation logic from before

# Initialize a dictionary to keep track of metabolites, their compartments, and reactions
metabolites_compartments = {}

for i, is_dead_end in enumerate(dead_ends):
    if is_dead_end:
        met = model.metabolites[i]
        base_id = met.id[:-2]  # Extract the base ID of the metabolite
        compartment = met.id[-1]  # Extract the compartment
        reactions = {str(met_rxn) for met_rxn in met.reactions}  # Use a set for unique reactions

        # Check if the base ID is already in the dictionary
        if base_id in metabolites_compartments:
            # Add the compartment if not already present and update the reactions set
            metabolites_compartments[base_id]['compartments'].add(compartment)
            metabolites_compartments[base_id]['reactions'].update(reactions)
        else:
            # If the base ID is not in the dictionary, add it with the current compartment and reactions
            metabolites_compartments[base_id] = {'compartments': {compartment}, 'reactions': reactions}

# Filtering metabolites present on opposite sides of reaction formulas
for metabolite, info in list(metabolites_compartments.items()):
    if len(info['compartments']) > 1:
        # Only keep metabolites that appear on opposite sides of the reaction equations
        if not check_metabolite_sides(info['reactions'], metabolite):
            del metabolites_compartments[metabolite]  # Remove metabolites not meeting the criteria

# Displaying the filtered results
counter=0
transport_reactions = []
for metabolite, info in metabolites_compartments.items():
    if len(info['compartments']) > 1:
        compartments = list(info['compartments'])
        for i in range(len(compartments)):
            for j in range(i+1, len(compartments)):
                # Construct the reaction string
                treaction = f"{metabolite}_{compartments[i]} <=> {metabolite}_{compartments[j]}"
                transport_reactions.append(treaction)
        print(f"{metabolite} is present in compartments: {', '.join(info['compartments'])}")
        print("Associated reactions:")
        for reaction in info['reactions']:
            print(reaction)
        print('------------------------------')
        print(f'Reaction created: {treaction}')
        counter+=1
        print()
print(counter)

In [ ]:
#Load models to extract boundary reactions from
iCHO1766 = read_sbml_model('../Data/Reconciliation/models/iCHOv1_final.xml')
iCHO2291 = read_sbml_model('../Data/Reconciliation/models/iCHO2291.xml')
recon3d = load_matlab_model('../Data/Reconciliation/models/Recon3D_301.mat')

models = [iCHO1766, iCHO2291, recon3d]

In [ ]:
###### --- Exctracting boundary reaction from other recosntructions --- ######

# Initialize the DataFrame with the desired columns
columns = ['ID','Name','Reaction', 'GPR', 'Subsystem', 'Lower Bound', 'Upper Bound']
reactions_df = pd.DataFrame(columns=columns)

# Initialize a set to track unique reaction IDs
seen_ids = set()

c=0
for mdl in models:
    for rxn in mdl.demands:
        # Standarize r.id according to our reconstruction
        rxn_id = rxn.id.replace('[', '_').replace(']', '').replace('_hs_', '_cho_').rstrip('_')
        if rxn_id not in seen_ids:
            seen_ids.add(rxn_id)
            # Standarize r.reaction according to our reconstruction
            rxn_reaction = rxn.reaction.replace('[', '_').replace(']', '').replace('_hs_', '_cho_')
            # Standarize met ids according to our reconstruction
            r_m = [m.id for m in rxn.metabolites][0].replace('[', '_').replace(']', '').replace('_hs_', '_cho_')
            for i, is_dead_end in enumerate(dead_ends):
                if is_dead_end:
                    met = model.metabolites[i]
                    if met.id == r_m:
                        # Create a temporary DataFrame for the new entry
                        new_row = pd.DataFrame({
                            'ID': [rxn_id],
                            'Name': [rxn.name],
                            'Reaction': [rxn_reaction],
                            'GPR': [rxn.gpr],
                            'Subsystem': [rxn.subsystem],
                            'Lower Bound': [rxn.lower_bound],
                            'Upper Bound': [rxn.upper_bound]
                        })
                        # Concatenate the new row to the main DataFrame
                        reactions_df = pd.concat([reactions_df, new_row], ignore_index=True)
                        c+=1

reactions_df.to_excel('temp/gap_fill_boundaries_output.xlsx', index=False)  # 'index=False' avoids writing row indices to the file.
print(f"Total reactions processed: {c}")

In [ ]:
###### --- Exctracting actual reactions from other reconstructions --- ######

# Initialize the DataFrame with the desired columns
columns = ['ID','Name','Reaction', 'GPR', 'Subsystem', 'Lower Bound', 'Upper Bound']
reactions_df = pd.DataFrame(columns=columns)

# Initialize a set to track unique reaction IDs
iCHO3000_rxn_ids = set([r.id for r in model.reactions])

c=0
for mdl in models:
    for rxn in mdl.reactions:
        # Standarize r.id according to our reconstruction
        rxn_id = rxn.id.replace('[', '_').replace(']', '').replace('_hs_', '_cho_').rstrip('_')
        if rxn_id not in iCHO3000_rxn_ids:
            iCHO3000_rxn_ids.add(rxn_id)
            # Standarize r.reaction according to our reconstruction
            rxn_reaction = rxn.reaction.replace('[', '_').replace(']', '').replace('_hs_', '_cho_')
            # Standarize met ids according to our reconstruction
            r_m = [m.id.replace('[', '_').replace(']', '').replace('_hs_', '_cho_') for m in rxn.metabolites]
            for i, is_dead_end in enumerate(dead_ends):
                if is_dead_end:
                    met = model.metabolites[i]
                    if met.id in r_m:
                        # Create a temporary DataFrame for the new entry
                        new_row = pd.DataFrame({
                            'ID': [rxn_id],
                            'Name': [rxn.name],
                            'Reaction': [rxn_reaction],
                            'GPR': [rxn.gpr],
                            'Subsystem': [rxn.subsystem],
                            'Lower Bound': [rxn.lower_bound],
                            'Upper Bound': [rxn.upper_bound],
                            'Dead_end': [met.id],
                            'Model': [mdl.id],
                        })
                        # Concatenate the new row to the main DataFrame
                        reactions_df = pd.concat([reactions_df, new_row], ignore_index=True)
                        c+=1

reactions_df.to_excel('temp/gap_fill_boundaries_output.xlsx', index=False)  # 'index=False' avoids writing row indices to the file.
print(f"Total reactions processed: {c}")

### 2.3 Addition of Extracellular Exchange Reanctions
The following cell adds **EXTRACELLULAR EXCHANGE** reactions to the dead-end metabolites in the extracellular compartment from the list generated above.

In [ ]:
##### ----- Automatically add EXTRACELLULAR EXCHANGE reactions to the "BoundaryRxns" Sheet ----- #####
added_exchange = False
for i,j in enumerate(dead_ends):
    if j:
        if str(model.metabolites[i]).endswith('_e'):
            new_row_data = {'Curated': '', 'Reaction': 'EX_'+str(model.metabolites[i]), 'Reaction Name': 'Exchange of '+model.metabolites[i].name, 'Reaction Formula': str(model.metabolites[i])+' <=>', 'Subsystem': 'EXTRACELLULAR EXCHANGE',
                                    'Reversible': 1, 'Lower bound': -1000, 'Upper bound': 1000, 'Objective': 0}
            new_row_df = pd.DataFrame(new_row_data, index=[len(boundary_rxns)])
            boundary_rxns = pd.concat([boundary_rxns, new_row_df])
            added_exchange = True

#Check for duplicated reactions added to the boundary_rxns dataset, IF NOT: update the google sheet file
if added_exchange:
    if not boundary_rxns['Reaction'].duplicated().any() and not boundary_rxns['Reaction Formula'].duplicated().any():
        sheet.update_google_sheet(sheet_boundary, boundary_rxns)
        print("BoundaryRxns Google Sheet updated.")
    else:
        print('Duplicated values found in the dataset')

### 2.4 Gapfill for blocked reactions
Cobrapy has a gap filling implementation that is very similar to that of Reed et al. where we use a mixed-integer linear program to figure out the smallest number of reactions that need to be added for a user-defined collection of reactions, i.e. a universal model.

In [ ]:
import cobra
from cobra.flux_analysis import gapfill

#recon_3d = read_sbml_model("../Data/GPR_curation/Recon3D.xml")
#iCHO2291 = read_sbml_model("../Data/Reconciliation/models/iCHO2291.xml")
#universal = recon_3d.merge(iCHO2291)

In [ ]:
for blocked_reaction in blocked:
    model.objective = blocked_reaction
    model.optimize().objective_value
    try:
        solution = gapfill(model, iCHO2291, demand_reactions=True)
        print(blocked_reaction)
        print(solution)
    except Exception as e:
        print(f'Gapfill failed for {blocked_reaction}: {str(e)}')
        continue

### Test CHO - Recon GEM

In [ ]:
universal

In [ ]:
# iCHO_recon3dfrom cobra.io import read_sbml_model
# read_sbml_model(".xml")

model_EX = [i for i, rxn in enumerate(model.reactions) if 'EX_' in rxn.id]
model_SK = [i for i, rxn in enumerate(model.reactions) if 'SK_' in rxn.id]
model_DM = [i for i, rxn in enumerate(model.reactions) if 'DM_' in rxn.id]
for i in model_EX:
    model.reactions[i].bounds = -1000, 1000

for i in model_SK:
    model.reactions[i].bounds = -1000, 1000

for i in model_DM:
    model.reactions[i].bounds = 0, 1000
    

In [ ]:
model.objective = "biomass_cho" # 
sol1 = model.optimize()
print(sol1.objective_value)

model.objective = "biomass_cho_prod" # 
sol2 = model.optimize()
print(sol2.objective_value)

In [ ]:
##### ----- Test model KOs ----- #####
for reaction in model.reactions:
    with model as model:
        reaction.knock_out()
        model.optimize()
        print('%s blocked (bounds: %s), new growth rate %f' %
              (reaction.id, str(reaction.bounds), model.objective.value))

## 3. Generation of the Mass Flow Graph <a id='MFG'></a>
Based on the publication **_Flux-dependent graphs for metabolic networks_** by _Beguerisse-Diaz et al. (2018)_ (https://www.nature.com/articles/s41540-018-0067-y). Here we use our model to build the **D Matrix** and plot the corresponding graph, then the **M Matrix** and plot the corresponding graph, and finally we generate the **PageRank** file with all the reactions in our reconstruction sorted by importance.

In [ ]:
# Import Libraries
import os
import webbrowser
import numpy as np
import pandas as pd
import networkx as nx
from scipy.linalg import pinv
from scipy.sparse import csr_matrix

import time

from dash import Dash, html
import dash_cytoscape as cyto
from skimage import draw
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from wordcloud import WordCloud
from collections import Counter

import cobra
from cobra.io import load_json_model

In [ ]:
##### ----- Read Model ----- #####
if 'model' not in locals():
    model = load_json_model("iCHOv3_CHO_15032024.json")
    print('model loaded')

### 3.1 Generation of the "D-Matrix"
The "D-Matrix" defines the weight of the edge between reaction nodes Ri and Rj as the probability that any metabolite chosen at random is produced by Ri (reaction i) and consumed by Rj (reaction j). Summing over all metabolites and normalizing, we obtain the edge weights of the adjacency matrix of the NFG:

In [ ]:
# Extract stoichiometric matrix, m=reactions, n=metabolites
start = time.time()
S = cobra.util.array.create_stoichiometric_matrix(model)
n, m = S.shape
end = time.time()
print(f"Time taken to generate S, n and m: {end - start} seconds")

# Create S2m matrix
start = time.time()
r = np.array([reaction.reversibility for reaction in model.reactions]) # m-dimensional reversibility vector with components rj = 1 if reaction Rj is reversible and rj = 0 if it is irreversible.
Im = np.eye(m) # m × m identity matrix
S2m = np.hstack((S, - S * r)) # unfolded version of the stoichiometric matrix of the 2m forward and reverse reactions.
end = time.time()
print(f"Time taken to generate S2m: {end - start} seconds")

# Create S2mplus and S2mminus matrices
start = time.time()
S2mplus = (np.abs(S2m) + S2m) / 2 # production stoichiometric matrix
S2mminus = (np.abs(S2m) - S2m) / 2 # consumption stoichiometric matrix
end = time.time()
print(f"Time taken to generate S2mplus and S2minus: {end - start} seconds")

# Calculate weights
start = time.time()
Wplus = np.diag(np.nan_to_num(1/ np.sum(S2mplus, axis = 1)))
Wminus = np.diag(np.nan_to_num(1 / np.sum(S2mminus, axis = 1)))
end = time.time()
print(f"Time taken to generate Wplus and Wminus: {end - start} seconds")

# Calculate D matrix
start = time.time()
D = 1/n * (Wplus @ S2mplus).T @ (Wminus @ S2mminus)
end = time.time()
print(f"Time taken to generate the D-Matrix: {end - start} seconds")
'''
# Remove unused reactions
start = time.time()
IDr = np.nonzero(np.sum(abs(D), axis=0) + np.sum(abs(D), axis=1) == 0)[0]
#IDr = ( np.sum(abs(D), axis=0) + np.sum(abs(D), axis=1) ) == 0
IDr = IDr[IDr > m]

D = np.delete(D, IDr, axis=0)
D = np.delete(D, IDr, axis=1)
end = time.time()
print(f"Time taken to remove unused reactions from the D-Matrix: {end - start} seconds")
'''

### 3.2 Plotting the "D-Matrix", Normalized Flow Graph (NFG)
The NFG is a weighted, directed graph with reactions as nodes, the edges represent supplier-consumer relationships between reactions, and weights given by the probability that a metabolite chosen at random from all reactions is produced/consumed by the source/target reaction (this discounts naturally the over-representation of pool metabolites). The edge indicates that metabolites are produced by the source reaction and consumed by the target reaction, thus accounting for metabolic directionality.

In [ ]:
# -------------------
# NetworkX Processing
# -------------------

# Convert D matrix into a graph
G = nx.from_numpy_array(D, create_using=nx.DiGraph)

# Filter nodes based on degree
min_degree = 2500
degrees = dict(G.degree())
nodes_to_keep = [node for node, degree in degrees.items() if degree >= min_degree]
G_filtered = G.subgraph(nodes_to_keep)

# Normalize weights for visualization
weights = [G_filtered[u][v]['weight'] for u, v in G_filtered.edges()]
min_weight, max_weight = min(weights), max(weights)
normalized_weights = [(w - min_weight) / (max_weight - min_weight) * (7 - 0.001) + 0.001 for w in weights]

# Mapping node indices to reaction names
reaction_names = np.concatenate(([r.id for r in model.reactions], [r.id + '_r' for r in model.reactions]))
node_labels_filtered = {i: reaction_name for i, reaction_name in enumerate(reaction_names) if i in nodes_to_keep}

# Color mapping based on degree
degrees_filtered = [val for (node, val) in G_filtered.degree()]
min_deg, max_deg = min(degrees_filtered), max(degrees_filtered)
normalized_degrees_filtered = [(d - min_deg) / (max_deg - min_deg) for d in degrees_filtered]
cmap = plt.get_cmap('OrRd')
node_colors_filtered = [cmap(deg) for deg in normalized_degrees_filtered]

In [ ]:
# -------------
# Matplotlib Plot
# -------------

fig, ax = plt.subplots(figsize=(20, 20))
pos_filtered = nx.random_layout(G_filtered)
nx.draw(G_filtered, pos_filtered, width=normalized_weights, node_color=node_colors_filtered, edge_color='lightgray', node_size=300, ax=ax, edgecolors='black', linewidths=0.5)
plt.title('Network Flux Graph (NFG)', fontsize=25, y=0.95)

# Adding labels and colorbar
label_pos_filtered = {node: (pos[0] + 0.012, pos[1] + 0.012) for node, pos in pos_filtered.items()}
nx.draw_networkx_labels(G_filtered, label_pos_filtered, labels=node_labels_filtered, font_size=10, ax=ax)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=min(normalized_degrees_filtered), vmax=max(normalized_degrees_filtered)))
plt.colorbar(sm, ax=ax, orientation='horizontal', label='Node Degree')

plt.savefig('../Networks/normalized_flow_graph.png', bbox_inches='tight', pad_inches=0.1)
plt.show()

In [ ]:
# ----------------
# Dash Cytoscape Setup
# ----------------

# Preparing elements for Dash Cytoscape
elements = [
    {
        'data': {'id': str(node), 'label': node_labels_filtered[node]},
        'position': {'x': pos[0] * 1000, 'y': pos[1] * 1000}
    }
    for node, pos in pos_filtered.items()
] + [
    {'data': {'id': f'{u}-{v}', 'source': str(u), 'target': str(v)}}
    for u, v in G_filtered.edges()
]

# Convert RGBA to hexadecimal for Dash
hex_colors = [mcolors.to_hex(color) for color in node_colors_filtered]

stylesheet = []

# Node style with individual colors
for i, node in enumerate(G_filtered.nodes()):
    hex_color = hex_colors[i]  # Get the hexadecimal color for the node
    node_style = {
        'selector': f'[id = "{str(node)}"]',
        'style': {
            'background-color': hex_color,
            'label': 'data(label)',
            'color': 'black',  # Adjust label color as needed
            'border-color': 'black',
            'border-width': 1,
        }
    }
    stylesheet.append(node_style)

# Edge styles remain the same
stylesheet += [
    {
        'selector': 'edge',
        'style': {
            'width': 1,
            'line-color': '#888'
        }
    }
]


# Dash app initialization and layout setup
app = Dash(__name__)
app.layout = html.Div([
    html.P("Interactive Metabolic Network:"),
    cyto.Cytoscape(id='network', elements=elements, style={'width': '1000px', 'height': '1000px'}, layout={'name': 'preset'}, stylesheet=stylesheet)
])

# Running the Dash app
if __name__ == '__main__':
    port = 8051  # Ensure this port is free or adjust as needed
    url = f'http://127.0.0.1:{port}/'
    webbrowser.open_new_tab(url)
    app.run_server(debug=True, port=port)

### 3.3 Generation of the "FluxOpenValue" matrix

In [ ]:
# Fix the bounds for boundary reactions
model_EX = [i for i, rxn in enumerate(model.reactions) if 'EX_' in rxn.id]
model_SK = [i for i, rxn in enumerate(model.reactions) if 'SK_' in rxn.id]
model_DM = [i for i, rxn in enumerate(model.reactions) if 'DM_' in rxn.id]
for i in model_EX:
    model.reactions[i].bounds = -1000, 1000

for i in model_SK:
    model.reactions[i].bounds = -1000, 1000

for i in model_DM:
    model.reactions[i].bounds = 0, 1000

# Perform pFBA for Biomass on Non-Producing and Producing Cell Lines
model.solver = 'gurobi'
objectives = ['biomass_cho', 'biomass_cho_prod']

fluxes_list = []
for objective in objectives:
    model.objective = objective
    pfba_solution = cobra.flux_analysis.pfba(model)
    fluxes = np.array(pfba_solution.fluxes)
    fluxes_list.append(fluxes)
    
# Stack arrays horizontally
FluxOpenValue = np.column_stack(fluxes_list)

FluxOpenValue

In [ ]:
# Count the number of nonzero values in each row
nonzero_counts = np.count_nonzero(FluxOpenValue, axis=1)

# Count the number of rows that contain only zeros
num_all_zero_rows = np.count_nonzero(nonzero_counts == 0)

# Count the number of rows that contain some nonzero value
num_some_nonzero_rows = np.count_nonzero(nonzero_counts != 0)

print("Number of Rxns with no flux:", num_all_zero_rows)
print("Number of Rxns with any flux:", num_some_nonzero_rows)

### 3.4 Generation of the "Mass Flow Graph (MFG) Matrix"
The MFG is a directed, environment-dependent, graph with weights computed from Flux Balance Analysis (FBA)

In [ ]:
# Calculation of the MFG for each pFBA analysis
M_list = []
for i in range(len(objectives)):
    start = time.time()
    v1 = FluxOpenValue[:, i].T

    # unfolding the flux vector
    # creation of vplus and vminus
    vplus = (np.abs(v1) + v1) / 2
    vminus = (np.abs(v1) - v1) / 2

    # creation of v2m
    v2m = np.concatenate((vplus, vminus))

    # creation of J_v
    J_v = S2mplus @ v2m.reshape(-1)

    # calculation of the MFG
    M = (S2mplus * v2m).T @ np.diag(np.nan_to_num(1/J_v)) @ (S2mminus * v2m)
    
    # Dynamically create a variable named M_<objective>
    objective_name = objectives[i]
    globals()[f'M_Matrix_{objective_name}'] = M

    end = time.time()
    print(f"Time taken to go through iteration {i+1}: {end - start} seconds")
    
'''
# Post-processing of PageRank
df = pd.DataFrame(PageRank)
PageRank = df.values
PageRank = np.array(PageRank).T
PageRankRxns = PageRank[:m, :]
PageRankRxns_back = PageRank[m:, :]

for i in range(m):
    for j in range(PageRankRxns.shape[1]):
        if PageRankRxns_back[i, j] > PageRankRxns[i, j]:
            PageRankRxns[i, j] = PageRankRxns_back[i, j]
'''

### 3.5 Plotting the "MFG Matrix"

In [ ]:
# -------------------
# NetworkX Processing
# -------------------

# Convert M matrix into a graph
G = nx.from_numpy_array(M_Matrix_biomass_cho, create_using=nx.DiGraph)

# Filter nodes based on degree and specific reactions to keep
min_degree = 1
degrees = {node: val for (node, val) in G.degree()}
nodes_to_keep = [node for node, degree in degrees.items() if degree >= min_degree]

# Additional logic to ensure specific reactions are always included
reactions_to_keep = ["LipidSyn", "DNAsyn", "RNAsyn", "PROTsyn", "biomass_cho"]
reaction_names = np.concatenate(([reaction.id for reaction in model.reactions], 
                                 [reaction.id + '_r' for reaction in model.reactions]))
indices_to_keep = [i for i, reaction_name in enumerate(reaction_names) if reaction_name in reactions_to_keep]
nodes_to_keep = list(set(nodes_to_keep).union(set(indices_to_keep)))

# Mapping node indices to reaction names for labeling
node_labels_filtered = {i: reaction_name for i, reaction_name in enumerate(reaction_names) if i in nodes_to_keep}

# Creating a subgraph and relabeling nodes
G_filtered = G.subgraph(nodes_to_keep)
G_filtered = nx.relabel_nodes(G_filtered, node_labels_filtered)

In [ ]:
# -----------------
# Matplotlib Plot
# -----------------

# Normalize weights and degrees for edge width and node color
weights = [G_filtered[u][v]['weight'] for u, v in G_filtered.edges()]
normalized_weights = [(w - min(weights)) / (max(weights) - min(weights)) * (7 - 0.001) + 0.001 for w in weights]
degrees_filtered = [val for (node, val) in G_filtered.degree()]
normalized_degrees_filtered = [(d - min(degrees_filtered)) / (max(degrees_filtered) - min(degrees_filtered)) for d in degrees_filtered]
cmap = plt.get_cmap('OrRd')
node_colors_filtered = [cmap(deg) for deg in normalized_degrees_filtered]

# Plotting with Matplotlib
fig, ax = plt.subplots(figsize=(20, 20))
pos_filtered = nx.random_layout(G_filtered)
nx.draw(G_filtered, pos_filtered, width=normalized_weights, node_color=node_colors_filtered, edge_color='lightgray', 
        node_size=300, ax=ax, edgecolors='black', linewidths=0.5)
plt.title('Mass Flow Graph (MFG)', fontsize=20, y=0.95)

# Adjusting labels and adding colorbar
offset = 0.02  # Adjust this value to move the labels up
pos_labels = {node: (x, y + offset) for node, (x, y) in pos_filtered.items()}
nx.draw_networkx_labels(G_filtered, pos_labels, ax=ax, font_size=10)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=min(normalized_degrees_filtered), vmax=max(normalized_degrees_filtered)))
plt.colorbar(sm, ax=ax, orientation='horizontal', label='Node Degree')
plt.savefig('../Networks/mass_flow_graph.png', bbox_inches='tight', pad_inches=0.1)
plt.show()

In [ ]:
# ----------------
# Dash Cytoscape Setup
# ----------------

# Function to map node degrees to color values
def degree_to_color(degree, cmap=plt.get_cmap('OrRd')):
    max_degree = max(dict(G_filtered.degree()).values())
    min_degree = min(dict(G_filtered.degree()).values())
    return cmap((degree - min_degree) / (max_degree - min_degree))

# Function to adjust edge width based on its weight
def adjust_weight(weight):
    min_weight = min([G_filtered[u][v]['weight'] for u, v in G_filtered.edges()])
    max_weight = max([G_filtered[u][v]['weight'] for u, v in G_filtered.edges()])
    min_edge_width = 0.5  # Adjusted for clearer visibility
    max_edge_width = 10
    normalized_weight = (weight - min_weight) / (max_weight - min_weight)
    return normalized_weight * (max_edge_width - min_edge_width) + min_edge_width

# Generate elements for nodes and edges
elements = []
stylesheet = [
    {'selector': 'edge', 'style': {'line-color': '#CCCCCC'}},
]

for node, data in G_filtered.nodes(data=True):
    degree = G_filtered.degree(node)
    color_hex = matplotlib.colors.to_hex(degree_to_color(degree))
    elements.append({'data': {'id': str(node), 'label': str(node)}})
    stylesheet.append({
        'selector': f'#{node}',
        'style': {
            'background-color': color_hex,
            'label': 'data(label)',
            'color': 'black',
            'border-color': 'black',
            'border-width': 1
        }
    })

for source, target, data in G_filtered.edges(data=True):
    adjusted_weight = adjust_weight(data['weight'])
    edge_id = f'{source}-{target}'
    elements.append({
        'data': {'id': edge_id, 'source': str(source), 'target': str(target)}
    })
    stylesheet.append({
        'selector': f'#{edge_id}',
        'style': {'width': adjusted_weight}
    })

# Setup Dash app
app = Dash(__name__)
app.layout = html.Div([
    html.P("Interactive Network Visualization:"),
    cyto.Cytoscape(
        id='network',
        elements=elements,
        layout={'name': 'random'},
        style={'width': '800px', 'height': '800px'},
        stylesheet=stylesheet
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)

### 3.6 Word Cloud Plot for Metabolites Frecuencies in Pagerank

In [ ]:
# Calculate and store PageRank
PageRank = []
G = nx.from_numpy_array(M_Matrix, create_using=nx.DiGraph)
pr = nx.pagerank(G)
PageRank.append(pr)

In [ ]:
# Post-processing of PageRank
S = cobra.util.array.create_stoichiometric_matrix(model)
n, m = S.shape
df = pd.DataFrame(PageRank)
PageRank = df.values
PageRank = np.array(PageRank).T
PageRankRxns = PageRank[:m, :]
PageRankRxns_back = PageRank[m:, :]

for i in range(m):
    for j in range(PageRankRxns.shape[1]):
        if PageRankRxns_back[i, j] > PageRankRxns[i, j]:
            PageRankRxns[i, j] = PageRankRxns_back[i, j]

In [ ]:
#row_sums = PageRankRxns.sum(axis=1)
df = pd.DataFrame()
for i,n in enumerate(objectives):
    sorted_indices = np.argsort(PageRankRxns[:,i])
    rxns_list = []
    values_list = []
    for s in sorted_indices[::-1]:
        rxns_list.append(model.reactions[s].id)
        values_list.append(PageRankRxns[s,i])
    
    df[n] = pd.Series(rxns_list)
    df[f'values_{n}'] = pd.Series(values_list)

In [ ]:
for i,v in df.iterrows():
    print(v['biomass_producing'],v['values_biomass_producing'])

In [ ]:
mets_list = []
for rxn in df['biomass'][df['values_biomass'] > 0.0000412]:
    r = model.reactions.get_by_id(rxn)
    mets = r.metabolites
    for met in mets:
        mets_list.append(met.id)
        
for rxn in df['biomass_producing'][df['values_biomass_producing'] > 0.0000412]:
    r = model.reactions.get_by_id(rxn)
    mets = r.metabolites
    for met in mets:
        mets_list.append(met.id)

In [ ]:
# Count the frequencies of each metabolite
mets_freq = Counter(mets_list)
mets_freq = {k: v for k, v in mets_freq.items() if not k.startswith('h2o_')} #eliminate water
mets_freq = {k: v for k, v in mets_freq.items() if not k.startswith('h2o2_')} #eliminate peroxide
mets_freq = {k: v for k, v in mets_freq.items() if not k.startswith('co2_')} #eliminate carbon dioxide
mets_freq = {k: v for k, v in mets_freq.items() if not k.startswith('nh4_')} #eliminate amonium
mets_freq = {k: v for k, v in mets_freq.items() if not k.startswith('h_')} #eliminate protons
mets_freq = {k: v for k, v in mets_freq.items() if not k.startswith('atp_')} #eliminate atp
mets_freq = {k: v for k, v in mets_freq.items() if not k.startswith('adp_')} #eliminate adp
mets_freq = {k: v for k, v in mets_freq.items() if not k.startswith('amp_')} #eliminate amp
mets_freq = {k: v for k, v in mets_freq.items() if not k.startswith('nad_')} #eliminate nad
mets_freq = {k: v for k, v in mets_freq.items() if not k.startswith('nadh_')} #eliminate nadh
mets_freq = {k: v for k, v in mets_freq.items() if not k.startswith('nadp_')} #eliminate nadp
mets_freq = {k: v for k, v in mets_freq.items() if not k.startswith('nadph_')} #eliminate nadph
mets_freq = {k: v for k, v in mets_freq.items() if not k.startswith('na1_')} #eliminate Sodium
mets_freq = {k: v for k, v in mets_freq.items() if not k.startswith('coa_')} #eliminate CoA
mets_freq = {k: v for k, v in mets_freq.items() if not k.startswith('accoa_')} #eliminate Acetyl-CoA
mets_freq = {k: v for k, v in mets_freq.items() if not k.startswith('pi_')} #eliminate phosphate
mets_freq = {k: v for k, v in mets_freq.items() if not k.startswith('ppi_')} #eliminate diphosphate
mets_freq = {k: v for k, v in mets_freq.items() if not k.startswith('fadh2_')} #eliminate FADH
mets_freq = {k: v for k, v in mets_freq.items() if not k.startswith('fad_')} #eliminate FAD
mets_freq = {k: v for k, v in mets_freq.items() if not k.startswith('o2_')} #eliminate Oxigen

# Create a circular mask
radius = 500  # you can change to the size you need
circle_img = np.zeros((2*radius, 2*radius), np.uint8)
rr, cc = draw.disk((radius, radius), radius)
circle_img[rr, cc] = 1

# Create the word cloud
wordcloud = WordCloud(width = 1000, height = 500, mask=circle_img, background_color="rgba(255, 255, 255, 0)", mode="RGBA").generate_from_frequencies(mets_freq)

plt.figure(figsize=(8,8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")

plt.savefig('wordcloud.png', bbox_inches='tight', transparent=True, pad_inches=0)
plt.show()

In [ ]:
counter=0
for met in mets_freq:
    print(met,mets_freq[met])
    counter+=1
    
print(counter)

In [ ]:
#Store the metabolites and their frequencies in a .txt file

with open('metabolites.txt', 'w') as f:
    for i, j in enumerate(mets_freq):
        print(j,'Freq:',mets_freq[j], file=f)

## 4. Biomass Conecting Reactions <a id='biomass'></a>

In [ ]:
import cobra
from cobra.flux_analysis import pfba
from cobra.io import load_json_model

import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
##### ----- Read Model ----- #####
if 'model' not in locals():
    model = load_json_model("iCHOv3_CHO_20022024.json")
    print('Model loaded')
else:
    print('Model already generated')

In [ ]:
model.solver = 'gurobi'
sol = pfba(model)

In [ ]:
sol

In [ ]:
ogpfba = sol.fluxes
duplicate_series = ogpfba.copy()
duplicate_series.index = [f"{idx}_r" for idx in duplicate_series.index]
finalpfba = pd.concat([ogpfba, duplicate_series])

In [ ]:
indexes_with_flux = []
rxn_with_flux = []
counter=0
for i,(r,f) in enumerate(ogpfba.items()):
    if f != 0:
        print(i,counter,r,f)
        indexes_with_flux.append(i)
        rxn_with_flux.append(r)
        counter+=1

In [ ]:
counter=0
for i,flux in enumerate(sol.fluxes):
    if flux != 0:
        r = sol.fluxes.index[i]
        #print(i,r,flux)
        counter+=1
    
print(counter)

In [ ]:
active_reactions = [reaction_id for reaction_id, flux in sol.fluxes.items() if flux != 0]

In [ ]:
active_reactions

In [ ]:
S = cobra.util.array.create_stoichiometric_matrix(model, array_type='DataFrame')
S_filtered = S[active_reactions]

In [ ]:
S_filtered

In [ ]:
import time
# Extract stoichiometric matrix, m=reactions, n=metabolites
start = time.time()
n, m = S_filtered.shape
end = time.time()
print(f"Time taken to generate S, n and m: {end - start} seconds")

# Create S2m matrix
start = time.time()
r = np.array([reaction.reversibility for reaction in model.reactions if reaction.id in S_filtered.columns]) # m-dimensional reversibility vector with components rj = 1 if reaction Rj is reversible and rj = 0 if it is irreversible.
Im = np.eye(m) # m × m identity matrix
S2m = np.hstack((S_filtered, - S_filtered * r)) # unfolded version of the stoichiometric matrix of the 2m forward and reverse reactions.
end = time.time()
print(f"Time taken to generate S2m: {end - start} seconds")

# Create S2mplus and S2mminus matrices
start = time.time()
S2mplus = (np.abs(S2m) + S2m) / 2 # production stoichiometric matrix
S2mminus = (np.abs(S2m) - S2m) / 2 # consumption stoichiometric matrix
end = time.time()
print(f"Time taken to generate S2mplus and S2minus: {end - start} seconds")

# Calculate weights
start = time.time()
Wplus = np.diag(np.nan_to_num(1/ np.sum(S2mplus, axis = 1)))
Wminus = np.diag(np.nan_to_num(1 / np.sum(S2mminus, axis = 1)))
end = time.time()
print(f"Time taken to generate Wplus and Wminus: {end - start} seconds")

# Calculate D matrix
start = time.time()
D = 1/n * (Wplus @ S2mplus).T @ (Wminus @ S2mminus)
end = time.time()
print(f"Time taken to generate the D-Matrix: {end - start} seconds")

In [ ]:
import math

In [ ]:
# Convert D matrix into a graph
G = nx.from_numpy_array(D, create_using=nx.DiGraph)

# Create a subgraph containing only the nodes with a degree greater than min_degree
degrees = {node: val for (node, val) in G.degree()} # Calculate the degrees of each node
min_degree = 10 # Define the minimum degree for a node to be kept.
nodes_to_keep = [node for node, degree in degrees.items() if degree >= min_degree]
G_filtered = G.subgraph(nodes_to_keep)

# Normalize weights for the edges of the nodes
weights = [G_filtered[u][v]['weight'] for u,v in G_filtered.edges()]
normalized_weights = [(w - min(weights)) / (max(weights) - min(weights)) * (7 - 0.001) + 0.001 for w in weights]

# Create a dictionary mapping node indices to reaction names for labeling
reaction_names = np.concatenate(([reaction for reaction in S_filtered.columns], 
                                 [reaction + '_r' for reaction in S_filtered.columns]))
node_labels_filtered = {i: reaction_name for i, reaction_name in enumerate(reaction_names) if i in nodes_to_keep}

# Normalize the degrees for color mapping
degrees_filtered = [val for (node, val) in G_filtered.degree()]
normalized_degrees_filtered = [(d - min(degrees_filtered)) / (max(degrees_filtered) - min(degrees_filtered)) for d in degrees_filtered]

# Use a colormap to map normalized degrees to colors
cmap = plt.get_cmap('OrRd')  # Choose a colormap here
node_colors_filtered = [cmap(deg) for deg in normalized_degrees_filtered]

# Plot the graph
fig, ax = plt.subplots(figsize=(20, 20))
pos_filtered = nx.random_layout(G_filtered)

# Find the node number for "biomass_cho"
biomass_cho_node_number = {v: k for k, v in node_labels_filtered.items()}.get('biomass_cho', None)

if biomass_cho_node_number is not None:
    # Position "biomass_cho" at the lower right corner
    pos_filtered[biomass_cho_node_number] = (1, 0)
    
    # Identify predecessors and filter based on edge weight
    connected_nodes = G_filtered.predecessors(biomass_cho_node_number)
    weight_threshold = 0.0001  # Define your weight threshold here
    filtered_predecessors = [node for node in connected_nodes if G_filtered[node][biomass_cho_node_number]['weight'] >= weight_threshold]
    
    # Calculate positions for filtered_predecessors
    radius = 0.05  # Distance from "biomass_cho" node, adjust as needed
    angle_increment = math.pi / len(filtered_predecessors)  # Adjust for semi-circle or full circle

    # Position filtered_predecessors in a semi-circle starting from upper left
    for i, node in enumerate(filtered_predecessors):
        angle = math.pi + (i * angle_increment)  # Adjust starting angle for upper left
        # Convert polar to Cartesian coordinates
        x = pos_filtered[biomass_cho_node_number][0] + radius * math.cos(angle)
        y = pos_filtered[biomass_cho_node_number][1] + radius * math.sin(angle)
        pos_filtered[node] = (x, y)
        
    # Now handle second-level predecessors for each node
    second_level_predecessors = list(G_filtered.predecessors(node))
    angle_increment = math.pi / max(len(second_level_predecessors), 1)  # Avoid division by zero

    for j, second_node in enumerate(second_level_predecessors):
        # Calculate angle for second-level predecessors
        angle = math.pi + (j * angle_increment)  # Adjust starting angle for upper left
        # Convert polar to Cartesian coordinates for second-level predecessors
        x = pos_filtered[node][0] + second_level_radius * math.cos(angle)
        y = pos_filtered[node][1] + second_level_radius * math.sin(angle)
        pos_filtered[second_node] = (x, y)


nx.draw(G_filtered, pos_filtered, width=normalized_weights, node_color=node_colors_filtered, edge_color='lightgray', node_size=300, ax=ax, edgecolors='black', linewidths=0.5)
plt.title('Network Flux Graph (NFG)', fontsize=25, y=0.95)

# Labels
label_pos_filtered = {node: (x + 0.012, y + 0.012) for node, (x, y) in pos_filtered.items()}
nx.draw_networkx_labels(G_filtered, label_pos_filtered, labels=node_labels_filtered, font_size=10, ax=ax)

plt.savefig('../Networks/normalized_flow_graph.png', bbox_inches='tight', pad_inches=0.1)

plt.show()

In [ ]:
filtered_predecessors

In [ ]:
# Assuming G_filtered is your directed graph and "biomass_cho" is the node of interest
biomass_cho_index = {label: index for index, label in node_labels_filtered.items()}.get('biomass_cho')

# Set a threshold for filtering based on weight
weight_threshold = 0.0001  # Example threshold, adjust as needed

# Initialize a list to hold labels of filtered predecessors
filtered_predecessor_labels = []

if biomass_cho_index is not None:
    # Get all predecessors of "biomass_cho"
    predecessors = list(G_filtered.predecessors(biomass_cho_index))
    
    # Filter predecessors based on edge weight
    for pred in predecessors:
        weight = G_filtered[pred][biomass_cho_index]['weight']
        if weight >= weight_threshold:
            # If the weight meets the threshold, add the predecessor's label to the list
            if pred in node_labels_filtered:
                filtered_predecessor_labels.append(node_labels_filtered[pred])

# Now, filtered_predecessor_labels contains the labels of the filtered predecessors
print("Filtered predecessor labels:", filtered_predecessor_labels)

In [ ]:
filtered_predecessors